# AstraForge Toolkit - DeepAgent Client Quickstart

Stream a reply from the DeepAgent API using the lightweight HTTP client.

Prereqs:
- Backend running at `http://localhost:8001` (e.g., `make backend-serve`)
- An API key (create via the in-app API Keys screen or `/api/api-keys/`)
- Package installed (`pip install astraforge-toolkit` or from PyPI)


## Notebook walk-through

1. Install the toolkit plus dotenv so local environment variables (API URL/key) can be loaded from this folder.
2. Load `.env` entries and build `BASE_URL`/`API_KEY`, raising early if the key is missing so the rest of the notebook can assume a valid session.
3. Instantiate `DeepAgentClient`, create a sandbox session, and print the session ID so you can reuse it in other examples.
4. The following cells connect to the DeepAgent API via the sandbox; use them as a launch point for streaming replies or plugging in your own chat loop once the session exists.


### Getting the most from this notebook

- Complete the install command once; rerun when dependencies change so the environment stays fresh.
- Keep `.env` synced with `ASTRA_FORGE_API_URL` and `ASTRA_FORGE_API_KEY` to avoid authentication failures.
- When the sandbox session is printed, copy the ID for other examples or shell-based calls.
- Treat the streaming cells as a launch point for your own chat loops or webhook integrations.


In [2]:
# Install the local package in editable mode (run from the examples/ folder)
%pip install astraforge-toolkit python-dotenv deepagents --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os

BASE_URL = os.getenv("ASTRA_FORGE_API_URL", "http://localhost:8001/api")
API_KEY = os.getenv("ASTRA_FORGE_API_KEY", "") # for local setup go to http://localhost:5174/app/api-keys

if not API_KEY:
    raise RuntimeError("Set ASTRA_FORGE_API_KEY in your environment")

In [5]:
from astraforge_toolkit import DeepAgentClient

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
sandbox_session = client.create_sandbox_session()
SANDBOX_SESSION_ID = sandbox_session.session_id
print(f"Sandbox session: {SANDBOX_SESSION_ID} (workspace: {sandbox_session.workspace_path})")

Sandbox session: 3f79b888-94f6-4fbc-90b2-02cc137fcb64 (workspace: /workspace)


In [6]:
%pip install langchain-ollama --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gpt-oss:20b",
    validate_model_on_init=True,
    temperature=0,
    model_kwargs={"think": "high"}
)

In [8]:
from deepagents import create_deep_agent

In [10]:
from astraforge_toolkit import SandboxBackend, sandbox_shell

def backend_factory(rt):
    return SandboxBackend(
        rt,
        base_url=BASE_URL,
        api_key=API_KEY,
        session_id=SANDBOX_SESSION_ID
    )

In [11]:
tools = [sandbox_shell]

In [12]:
deep_agent = create_deep_agent(
    model=llm,
    backend=backend_factory,
    tools=tools,
)

In [13]:
import uuid
THREAD_ID = uuid.uuid4().hex

In [14]:
RUN_CONFIG = {
    "thread_id": THREAD_ID,
    "configurable": {"sandbox_session_id": SANDBOX_SESSION_ID},
}

In [17]:
deep_agent.invoke(
    {"messages": [{"role": "user", "content": "create a todo list for making crepes. call the tool"}]},
    config=RUN_CONFIG,
)

{'messages': [HumanMessage(content='create a todo list for making crepes. call the tool', additional_kwargs={}, response_metadata={}, id='ce8afa5a-0048-47bb-8211-2adacac0cde8'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-01-07T20:41:16.546849471Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4712323039, 'load_duration': 310432586, 'prompt_eval_count': 4350, 'prompt_eval_duration': 291685787, 'eval_count': 475, 'eval_duration': 3864205269, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--8d16b5f0-b250-4402-b22a-f0ed71bedf42-0', tool_calls=[{'name': 'write_todos', 'args': {'todos': [{'description': 'Gather all ingredients: flour, eggs, milk, butter, salt, sugar, vanilla extract', 'status': 'pending'}, {'description': 'Whisk flour, eggs, and milk together to make a smooth batter', 'status': 'pending'}, {'description': 'Add melted butter, a pinch of salt, sugar, and vanilla t